# Trying GPT

In [1]:
import os, json, logging, asyncio
from typing import Optional, List, Any, Dict, Type
import httpx
from pydantic import BaseModel, Field, create_model, ValidationError
from openai import AsyncAzureOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel, function_tool
# If you use sessions later:
# from agents import SQLiteSession

# ────────────────────────────────────────────────────────────────────────────────
# Global logging
# ────────────────────────────────────────────────────────────────────────────────
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(name)s: %(message)s")
logger = logging.getLogger("kahua_tool")
logger.setLevel(logging.INFO)
handoff_log = logging.getLogger("agents.handoff")
handoff_log.setLevel(logging.INFO)

# ────────────────────────────────────────────────────────────────────────────────
# Azure OpenAI client
# ────────────────────────────────────────────────────────────────────────────────
azure_client = AsyncAzureOpenAI(
    api_key=os.environ["AZURE_KEY"],
    azure_endpoint=os.environ["AZURE_ENDPOINT"],
    api_version=os.environ["API_VERSION"],
)
# --- CONSTANTS (one source of truth) ---
KAHUA_BASIC_AUTH = os.getenv("KAHUA_BASIC_AUTH")
def _auth_header() -> str:
    if not KAHUA_BASIC_AUTH:
        raise RuntimeError("KAHUA_BASIC_AUTH not set")
    return KAHUA_BASIC_AUTH if KAHUA_BASIC_AUTH.strip().lower().startswith("basic ") \
           else f"Basic {KAHUA_BASIC_AUTH}"

# Per your note, the 'RFI' segment is always used; only the project {id} changes.
ACTIVITY_URL = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/{id}/apps/kahua_AEC_RFI/activities/run"
QUERY_URL = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/query?returnDefaultAttributes=true"

# --- FIX 1+2: Use ContractItem set; unify headers/URL; keep entityDef id=0 ---
@function_tool(strict_mode=False)
async def create_dummy_entity(
    entity_def: str,
    projectid: int | None = None,
    set_name: str = "ContractItem",  # <-- IMPORTANT
) -> dict:
    headers = {"Content-Type": "application/json", "Authorization": _auth_header()}
    pid = 0 if projectid is None else int(projectid)
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": set_name,
            "Flow": [{"PropertyName": "Iterate", "Set": set_name, "New": {}, "Existing": {}}]
        },
        "listIdentifier": 0,
        "listId": 0,
        "sets": [{
            "name": set_name,
            "entities": [{"id": 0, "entityDef": entity_def}]
        }],
    }
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=pid), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    # Helper re-used from your codebase:
    created = _extract_first_entity(body, set_name)
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body, "created_entity": created}

@function_tool(strict_mode=False)
async def update_entity_by_id(
    entity_def: str,
    entity_id: int,
    fields: Dict[str, Any] | None = None,
    projectid: int | None = None,
    set_name: str = "ContractItem",  # <-- IMPORTANT
) -> dict:
    headers = {"Content-Type": "application/json", "Authorization": _auth_header()}
    pid = 0 if projectid is None else int(projectid)
    entity = {"id": int(entity_id), "entityDef": entity_def}
    if fields:
        entity.update(dict(fields))
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": set_name,
            "Flow": [{"PropertyName": "Iterate", "Set": set_name, "New": {}, "Existing": {}}]
        },
        "listIdentifier": 0,
        "listId": 0,
        "sets": [{"name": set_name, "entities": [entity]}],
    }
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=pid), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

# --- FIX 3+4+6: keep ONE send_custom_item; dict-based; consistent auth/URL ---
@function_tool(strict_mode=False)
async def send_custom_item(entity: Dict[str, Any], projectid: int | None = None) -> dict:
    if not isinstance(entity, dict):
        return {"status": "error", "message": "entity must be a dict"}
    if not entity.get("entityDef"):
        return {"status": "error", "message": "entityDef is required"}
    entity = dict(entity)
    entity["id"] = 0  # required
    pid = projectid if projectid is not None else entity.get("ProjectId") or 0
    headers = {"Content-Type": "application/json", "Authorization": _auth_header()}
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": "ContractItem",
            "Flow": [{"PropertyName": "Iterate", "Set": "ContractItem", "New": {}, "Existing": {}}]
        },
        "sets": [{"name": "ContractItem", "entities": [entity]}],
    }
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=int(pid)), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

# --- FIX 5: make aliases robust (singular/plural/short) ---
ENTITY_ALIASES = {
    "project": "kahua_Project.Project",
    "punchlist": "kahua_AEC_PunchList.PunchListItem",
    "punch list": "kahua_AEC_PunchList.PunchListItem",
    "rfi": "kahua_AEC_RFI.RFI",
    "submittal": "kahua_AEC_Submittal.Submittal",
    "change order": "kahua_AEC_ChangeOrder.ChangeOrder",
    "field observation": "kahua_AEC_FieldObservation.FieldObservationItem",
    "field observations": "kahua_AEC_FieldObservation.FieldObservationItem",
    "fo": "kahua_AEC_FieldObservation.FieldObservationItem",
}

# --- OPTIONAL: helper to choose project id consistently ---
def _project_id_from(entity_or_id: Any) -> int:
    if isinstance(entity_or_id, int):
        return entity_or_id
    if isinstance(entity_or_id, dict):
        return int(entity_or_id.get("ProjectId") or 0)
    return 0


In [11]:
# --- Imports / setup -----------------------------------------------------------
import os, json, logging
from typing import Optional, List, Any, Dict, Type

import httpx
from pydantic import BaseModel, Field, create_model, ValidationError

try:
    # Agents SDK (your package)
    from agents import Agent, Runner, OpenAIChatCompletionsModel, SQLiteSession, function_tool
except Exception:
    # Fallback no-op decorator so notebook cells still run
    def function_tool(func=None, **_kwargs):
        return func if func else (lambda f: f)
    raise RuntimeError("The 'agents' package must be importable for this stack to run.")

from openai import AsyncAzureOpenAI

# --- Logging ------------------------------------------------------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(name)s: %(message)s")
logger = logging.getLogger("kahua_tool")
logger.setLevel(logging.INFO)
handoff_log = logging.getLogger("agents.handoff")
handoff_log.setLevel(logging.INFO)

# --- Azure OpenAI client -------------------------------------------------------
azure_client = AsyncAzureOpenAI(
    api_key=os.environ["AZURE_KEY"],
    azure_endpoint=os.environ["AZURE_ENDPOINT"],
    api_version=os.environ["API_VERSION"],
)

# --- Constants / URLs ----------------------------------------------------------
# Per your note: the 'RFI' segment is always used. Only the project {id} changes.
ACTIVITY_URL = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/{id}/apps/kahua_AEC_RFI/activities/run"
QUERY_URL    = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/query?returnDefaultAttributes=true"

KAHUA_BASIC_AUTH = os.getenv("KAHUA_BASIC_AUTH")

def _auth_header() -> str:
    if not KAHUA_BASIC_AUTH:
        raise RuntimeError("KAHUA_BASIC_AUTH not set")
    return KAHUA_BASIC_AUTH if KAHUA_BASIC_AUTH.strip().lower().startswith("basic ") \
           else f"Basic {KAHUA_BASIC_AUTH}"

# --- Small utils ---------------------------------------------------------------
def _extract_first_entity(data: Dict[str, Any], set_name: str) -> Dict[str, Any]:
    sets = data.get("sets") or []
    for s in sets:
        if s.get("name") == set_name:
            ents = s.get("entities") or []
            if ents:
                return ents[0]
    for key in ("entities", "items", "results"):
        if isinstance(data.get(key), list) and data[key]:
            return data[key][0]
    return {}

# Robust alias table (singular/plural/short)
ENTITY_ALIASES: Dict[str,str] = {
    "project": "kahua_Project.Project",
    "punchlist": "kahua_AEC_PunchList.PunchListItem",
    "punch list": "kahua_AEC_PunchList.PunchListItem",
    "rfi": "kahua_AEC_RFI.RFI",
    "submittal": "kahua_AEC_Submittal.Submittal",
    "change order": "kahua_AEC_ChangeOrder.ChangeOrder",
    "field observation": "kahua_AEC_FieldObservation.FieldObservationItem",
    "field observations": "kahua_AEC_FieldObservation.FieldObservationItem",
    "fo": "kahua_AEC_FieldObservation.FieldObservationItem",
}

# --- Pydantic models used by certain tools ------------------------------------
class PunchItem(BaseModel):
    subject: Optional[str] = Field(None, description="Short subject")
    description: Optional[str] = Field(None, description="Detailed description")
    projectid: Optional[int] = Field(0, description="Project id")
    auto_send: bool = Field(False, description="If true, the tool may send immediately")

class ProjectItem(BaseModel):
    id: Optional[str] = Field(0, description="Project id")
    name: str = Field(..., description="Project name (Kahua: Name)")
    description: Optional[str] = Field(None, description="Project description (Kahua: Description)")
    auto_send: bool = Field(True, description="If true, the tool may send immediately")


@function_tool
async def query_entity_def(entity_def: str) -> dict:
    """Fire a simple query on an entityDef to prove access / shape."""
    headers = {"Content-Type": "application/json", "Authorization": _auth_header()}
    payload = {"PropertyName": "Query", "EntityDef": entity_def}
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(QUERY_URL, headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    return {"status": "ok", "upstream_status": resp.status_code, "body": body}

# Dummy create → returns server-shaped entity (id + schema in echo)
@function_tool(strict_mode=False)
async def create_dummy_entity(
    entity_def: str,
    projectid: int | None = None,
    set_name: str = "ContractItem",
) -> dict:
    headers = {"Content-Type": "application/json", "Authorization": _auth_header()}
    pid = 0 if projectid is None else int(projectid)
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": set_name,
            "Flow": [{"PropertyName": "Iterate", "Set": set_name, "New": {}, "Existing": {}}]
        },
        "listIdentifier": 0,
        "listId": 0,
        "sets": [{"name": set_name, "entities": [{"id": 0, "entityDef": entity_def}]}],
    }
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=pid), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    created = _extract_first_entity(body, set_name)
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body, "created_entity": created}

# Re-post with same id to persist edits
@function_tool(strict_mode=False)
async def update_entity_by_id(
    entity_def: str,
    entity_id: int,
    fields: Dict[str, Any] | None = None,
    projectid: int | None = None,
    set_name: str = "ContractItem",
) -> dict:
    headers = {"Content-Type": "application/json", "Authorization": _auth_header()}
    pid = 0 if projectid is None else int(projectid)
    entity = {"id": int(entity_id), "entityDef": entity_def}
    if fields:
        entity.update(dict(fields))
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": set_name,
            "Flow": [{"PropertyName": "Iterate", "Set": set_name, "New": {}, "Existing": {}}]
        },
        "listIdentifier": 0,
        "listId": 0,
        "sets": [{"name": set_name, "entities": [entity]}],
    }
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=pid), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

# One dict-based “send any custom entity”
@function_tool(strict_mode=False)
async def send_custom_item(entity: Dict[str, Any], projectid: int | None = None) -> dict:
    if not isinstance(entity, dict):
        return {"status": "error", "message": "entity must be a dict"}
    if not entity.get("entityDef"):
        return {"status": "error", "message": "entityDef is required"}
    entity = dict(entity)
    entity["id"] = 0
    pid = projectid if projectid is not None else entity.get("ProjectId") or 0
    headers = {"Content-Type": "application/json", "Authorization": _auth_header()}
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": "ContractItem",
            "Flow": [{"PropertyName": "Iterate", "Set": "ContractItem", "New": {}, "Existing": {}}]
        },
        "sets": [{"name": "ContractItem", "entities": [entity]}],
    }
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=int(pid)), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

# Create Project directly (kept for completeness)
@function_tool
async def create_project_in_kahua(project: ProjectItem) -> dict:
    entity = {
        "id": 0,
        "hubPath": "kahua_Project.NoWorkflow\\Start",
        "entityDef": "kahua_Project.Project",
        "Name": project.name,
    }
    if project.description:
        entity["Description"] = project.description
    if project.id:
        entity["Id"] = project.id

    headers = {"Content-Type": "application/json", "Authorization": _auth_header()}
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": "ContractItem",
            "Flow": [{"PropertyName": "Iterate", "Set": "ContractItem", "New": {}, "Existing": {}}]
        },
        "sets": [{"name": "ContractItem", "entities": [entity]}],
    }
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=0), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {
            "status": "error",
            "upstream_status": resp.status_code,
            "upstream_content_type": ctype,
            "upstream_body": body,
        }
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

# Build Pydantic model dynamically from example-like JSON (for custom schemas)
def _pick_representative(lst: list) -> Any:
    for x in lst:
        if x is not None:
            return x
    return lst[0] if lst else None

def _infer_model_from_example(data: Any, name_hint: str = "Model") -> Type[BaseModel] | Any:
    if isinstance(data, bool):   return Optional[bool]
    if isinstance(data, int):    return Optional[int]
    if isinstance(data, float):  return Optional[float]
    if isinstance(data, str):    return Optional[str]
    if data is None:             return Optional[Any]
    if isinstance(data, list):
        rep = _pick_representative(data)
        if isinstance(rep, dict):
            sub = _infer_model_from_example(rep, f"{name_hint}Item")
            return Optional[List[sub]]  # type: ignore[name-defined]
        elif rep is None:
            return Optional[List[Any]]
        else:
            prim = _infer_model_from_example(rep, f"{name_hint}Item")
            return Optional[List[prim]]  # type: ignore[valid-type]
    if isinstance(data, dict):
        fields: Dict[str, tuple] = {}
        for k, v in data.items():
            sub_type = _infer_model_from_example(v, f"{name_hint}_{k.capitalize()}")
            fields[k] = (sub_type, None)
        return create_model(name_hint, **fields)  # type: ignore[arg-type]
    return Optional[Any]

def _sanitize_json_schema(obj: Any) -> Any:
    if isinstance(obj, dict):
        obj.pop("additionalProperties", None)
        for k, v in list(obj.items()):
            obj[k] = _sanitize_json_schema(v)
    elif isinstance(obj, list):
        return [_sanitize_json_schema(x) for x in obj]
    return obj

@function_tool(strict_mode=False)
async def create_pydantic_model_from_schema(
    schema_like: Dict[str, Any],
    model_name: str = "DynamicModel",
    sample: Optional[Dict[str, Any]] = None
) -> dict:
    try:
        ModelOrType = _infer_model_from_example(schema_like, model_name)
        if not isinstance(ModelOrType, type) or not issubclass(ModelOrType, BaseModel):
            Model = create_model(model_name, value=(ModelOrType, None))  # type: ignore[arg-type]
        else:
            Model = ModelOrType

        json_schema = _sanitize_json_schema(Model.model_json_schema())
        validated_sample = None
        errors: List[str] = []
        status = "ok"
        if sample is not None:
            try:
                inst = Model(**sample)
                validated_sample = inst.model_dump(by_alias=False, exclude_none=True)
            except ValidationError as ve:
                status = "validation_error"
                errors = [str(e) for e in ve.errors()]

        try:
            python_model_str = f"class {Model.__name__}(BaseModel): " + ", ".join(Model.__fields__.keys())  # type: ignore[attr-defined]
        except Exception:
            python_model_str = f"{Model.__name__}(BaseModel)"

        return {
            "status": status,
            "model_name": Model.__name__,
            "json_schema": json_schema,
            "python_model_str": python_model_str,
            "validated_sample": validated_sample,
            "errors": errors,
        }
    except Exception as e:
        return {
            "status": "error",
            "model_name": model_name,
            "json_schema": None,
            "python_model_str": None,
            "validated_sample": None,
            "errors": [repr(e)],
        }

# --- Agent wiring (AFTER tools exist) -----------------------------------------
entity_query_agent = Agent(
    name="Entity Query",
    handoff_description="Specialist agent for entity query",
    model=OpenAIChatCompletionsModel(model=os.environ["AZURE_DEPLOYMENT"], openai_client=azure_client),
    instructions=(
        "to query any entity, map alias → entity_def and call query_entity_def(entity_def).\n\n"
        "ALIAS TABLE (authoritative):\n" + "\n".join([f"- {k} → {v}" for k,v in ENTITY_ALIASES.items()])
    ),
    tools=[query_entity_def],
)

project_creation_agent = Agent(
    name="Project",
    handoff_description="Specialist agent for project creation",
    model=OpenAIChatCompletionsModel(model=os.environ["AZURE_DEPLOYMENT"], openai_client=azure_client),
    instructions="You help users create construction projects.",
    tools=[create_project_in_kahua],
)

punch_list_agent = Agent(
    name="Punch List",
    handoff_description="Specialist agent for punch list creation.",
    model=OpenAIChatCompletionsModel(model=os.environ["AZURE_DEPLOYMENT"], openai_client=azure_client),
    instructions="You help users create construction punch list items.",
    tools=[create_punch_list_item] if 'create_punch_list_item' in globals() else [],
)

custom_item_agent = Agent(
    name="Custom Item",
    handoff_description="Specialist agent for custom item creation",
    model=OpenAIChatCompletionsModel(model=os.environ["AZURE_DEPLOYMENT"], openai_client=azure_client),
    instructions=(
        "Strict Kahua creation policy:\n"
        "1) Do NOT ask the user for a schema.\n"
        "2) Map alias → entityDef using ENTITY_ALIASES.\n"
        "3) Call create_dummy_entity(entity_def, projectid if known) to obtain id + schema.\n"
        "4) Choose fields from user intent (at minimum set 'Subject' if appropriate).\n"
        "5) Call update_entity_by_id(entity_def, entity_id, fields, projectid) to persist edits.\n"
        "6) Return concise JSON with id, fields set, and any links."
    ),
    tools=[create_dummy_entity, update_entity_by_id, send_custom_item],
)

triage_agent = Agent(
    name="Agentic Chatbot",
    instructions=(
        "Chatbot instructions:\n"
        "Behave like chatGPT.\n"
        "You are a helpful chatbot with tools for a project management software company called Kahua.\n"
        "You use the tools to answer questions, create records, and perform other tasks.\n"
        "Your tools use the Kahua API to interact with the software.\n"
        "Short API overview:\n"
        "Every post/creation/update/delete is sent to this API endpoint: https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/<projectid>/apps/kahua_AEC_RFI/activities/run with basic auth and a JSON payload.\n"
        "Where <projectid> is the id of a particular project or 0, for a root creation. Do not be fooled by the kahua_AEC_RFI part, it is the same for all apps. This does not mean you are creating an RFI, its just a hardcoded endpoint.\n"
        "This is the JSON payload format for a creation:\n"
        "{\n"
        "    \"activity\": {\n"
        "        \"PropertyName\": \"Activity\",\n"
        "        \"Name\": \"Activity\",\n"
        "        \"Flow\": [\n"
        "        {\n"
        "            \"PropertyName\": \"Iterate\",\n"
        "            \"Set\": \"Activity\",\n"
        "            \"New\": {},\n"
        "            \"Existing\": {\n"
        "            }\n"
        "        }\n"
        "        ]\n"
        "    },\n"
        "    \"listIdentifier\": 0,\n"
        "    \"listId\": 0,\n"
        "    \"sets\": [\n"
        "        {\n"
        "        \"name\": \"Activity\",\n"
        "        \"entities\": [\n"
        "            {\n"
        "            \"id\": <id>,\n"
        "            \"entityDef\": <entity_def>,\n"
        "            }\n"
        "        ]\n"
        "        }\n"
        "    ]\n"
        "    }\n"
        "}\n"
        "The only thing you ever need to change in this template is the <entity_def> and <id> fields. id will be 0 when creating a new item, and will be the id of the existing item when updating. entity_def will be a Kahua entityDef of the item you are creating, such as kahua_AEC_RFI.FieldObservationItem.\n"
        "Upon posting this JSON to the endpoint, you will receive a response with the entire schema of the new item, including the id field.\n"
        "Router:\n"
        "- If the user asks to create a punch list for a named project, FIRST call find_project_id_by_name(name). "
        "  If found, hand off to Punch List with projectid.\n"
        "- If they want a project created, hand off to Project.\n"
        "- If they want to query a Kahua entity, hand off to Entity Query.\n"
        "- If they want a custom entity created (RFI, Field Observation, Submittal, Change Order, etc.), hand off to Custom Item.\n"
        "- If they ask to generate a schema or validate a sample, hand off to Schema Creation.\n"
        "- If the user asks to create/log/file/add a construction record (RFI, Submittal, Field Observation, Issue, etc.), "
        "hand off to Custom Item. Custom Item must resolve the entityDef automatically without asking the user.\n"
        "- Avoid repeated handoffs; perform at most one lookup and one create."
    ),
    model=OpenAIChatCompletionsModel(model=os.environ["AZURE_DEPLOYMENT"], openai_client=azure_client),
    handoffs=[punch_list_agent, project_creation_agent, entity_query_agent, custom_item_agent],
)

logger.info("Agents wired successfully.")


2025-10-20 14:48:38,330 INFO kahua_tool: Agents wired successfully.


In [10]:
from agents import Runner, SQLiteSession

session = SQLiteSession("demo_session")

prompt = (
    "Create a field observation about the dog I found biting a worker. "
    "Leave all other fields blank/null. No project id. "
    "You have the correct entity def."
)

# JUPYTER/NOTEBOOK: use the async API
result = await Runner.run(
    triage_agent,
    prompt,
    session=session,
)

print("FINAL OUTPUT:\n", result.final_output)


2025-10-20 14:35:49,657 INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-10-20 14:35:50,117 INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-10-20 14:35:50,582 INFO httpx: HTTP Request: POST https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/apps/kahua_AEC_RFI/activities/run "HTTP/1.1 400 Bad Request"
2025-10-20 14:35:50,951 INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-10-20 14:35:51,356 INFO httpx: HTTP Request: POST https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/apps/kahua_AEC_RFI/activities/run "HTTP/1.1 400 Bad Request"
2025-10-20 14:35:51,689 INFO httpx: HTTP Requ

FINAL OUTPUT:
 It seems the entity definition I used is invalid or unrecognized. Could you confirm or clarify the entity type you had in mind for this observation?


# SuperAgent 

In [3]:
# Kahua SuperAgent — query/create/update ANY entity
# Notebook-friendly: no argparse, runs as-is with your env + agents package.

import os, json, logging, asyncio
from typing import Optional, List, Any, Dict
import httpx
from pydantic import BaseModel, Field, ValidationError
from openai import AsyncAzureOpenAI


# ────────────────────────────────────────────────────────────────────────────────
# Logging
# ────────────────────────────────────────────────────────────────────────────────
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(name)s: %(message)s")
log = logging.getLogger("kahua_superagent")

# ────────────────────────────────────────────────────────────────────────────────
# Agents SDK (must exist in your env)
# ────────────────────────────────────────────────────────────────────────────────
try:
    from agents import Agent, Runner, OpenAIChatCompletionsModel, function_tool, SQLiteSession, WebSearchTool
except Exception as e:
    raise RuntimeError("The 'agents' package must be importable for this script to run.") from e

# ────────────────────────────────────────────────────────────────────────────────
# Azure OpenAI client
# ────────────────────────────────────────────────────────────────────────────────
azure_client = AsyncAzureOpenAI(
    api_key=os.environ["AZURE_KEY"],
    azure_endpoint=os.environ["AZURE_ENDPOINT"],
    api_version=os.environ["API_VERSION"],
)
MODEL_DEPLOYMENT = os.environ["AZURE_DEPLOYMENT"]

# ────────────────────────────────────────────────────────────────────────────────
# Kahua constants and auth helpers
# ────────────────────────────────────────────────────────────────────────────────
# NOTE: Per your stack, regardless of entity/app, we post to this RFI app endpoint.
ACTIVITY_URL = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/{id}/apps/kahua_AEC_RFI/activities/run"
QUERY_URL    = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/query?returnDefaultAttributes=true"

KAHUA_BASIC_AUTH = os.getenv("KAHUA_BASIC_AUTH")

def _auth_header_value() -> str:
    if not KAHUA_BASIC_AUTH:
        raise RuntimeError("KAHUA_BASIC_AUTH not set")
    return KAHUA_BASIC_AUTH if KAHUA_BASIC_AUTH.strip().lower().startswith("basic ") \
           else f"Basic {KAHUA_BASIC_AUTH}"

HEADERS_JSON = lambda: {"Content-Type": "application/json", "Authorization": _auth_header_value()}

# ────────────────────────────────────────────────────────────────────────────────
# Entity alias table (extend freely)
# ────────────────────────────────────────────────────────────────────────────────
ENTITY_ALIASES: Dict[str, str] = {
    # Core
    "project": "kahua_Project.Project",

    # AEC
    "rfi": "kahua_AEC_RFI.RFI",
    "submittal": "kahua_AEC_Submittal.Submittal",
    "change order": "kahua_AEC_ChangeOrder.ChangeOrder",
    "punchlist": "kahua_AEC_PunchList.PunchListItem",
    "punch list": "kahua_AEC_PunchList.PunchListItem",
    "field observation": "kahua_AEC_FieldObservation.FieldObservationItem",
    "field observations": "kahua_AEC_FieldObservation.FieldObservationItem",
    "fo": "kahua_AEC_FieldObservation.FieldObservationItem",

    # Media/attachments (example)
    "image": "kahua_Core.kahua_MediaImage",
}

def resolve_entity_def(name_or_def: str) -> str:
    key = (name_or_def or "").strip().lower()
    return ENTITY_ALIASES.get(key, name_or_def)

def _extract_first_entity(data: Dict[str, Any], set_name: str) -> Dict[str, Any]:
    sets = data.get("sets") or []
    for s in sets:
        if s.get("name") == set_name:
            ents = s.get("entities") or []
            if ents:
                return ents[0]
    for key in ("entities", "items", "results"):
        if isinstance(data.get(key), list) and data[key]:
            return data[key][0]
    return {}

# ────────────────────────────────────────────────────────────────────────────────
# Generic POST builder to activities/run
# ────────────────────────────────────────────────────────────────────────────────
async def _post_activity(entities: List[Dict[str, Any]], projectid: int = 0, set_name: str = "ContractItem") -> httpx.Response:
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": set_name,
            "Flow": [{"PropertyName": "Iterate", "Set": set_name, "New": {}, "Existing": {}}]
        },
        "listIdentifier": 0,
        "listId": 0,
        "sets": [{"name": set_name, "entities": entities}],
    }
    async with httpx.AsyncClient(timeout=25.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=int(projectid)), headers=HEADERS_JSON(), json=payload)
    return resp

# ────────────────────────────────────────────────────────────────────────────────
# Tools: Query / Create / Update / Upsert / SendRaw / FindProject
# ────────────────────────────────────────────────────────────────────────────────
@function_tool
async def query_entities(entity_def: str, limit: int = 50) -> dict:
    """
    Run a simple server-side query for an entity_def and return body.
    """
    ent = resolve_entity_def(entity_def)
    qpayload = {"PropertyName": "Query", "EntityDef": ent}
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(QUERY_URL, headers=HEADERS_JSON(), json=qpayload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    # Optionally trim results if huge
    if isinstance(body, dict):
        for k in ("entities", "results", "items"):
            if isinstance(body.get(k), list) and len(body[k]) > limit:
                body[k] = body[k][:limit]
    return {"status": "ok", "entity_def": ent, "body": body}

@function_tool(strict_mode=False)
async def create_entity(entity_def: str, fields: Dict[str, Any] | None = None, projectid: int = 0, set_name: str = "ContractItem") -> dict:
    """
    Create ANY entity with id=0 and optional initial fields.
    """
    ent = resolve_entity_def(entity_def)
    entity = {"id": 0, "entityDef": ent}
    if fields: entity.update(fields)
    resp = await _post_activity([entity], projectid=projectid, set_name=set_name)
    ctype = resp.headers.get("content-type", "")
    body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    created = _extract_first_entity(body, set_name)
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body, "created": created}

@function_tool(strict_mode=False)
async def update_entity(entity_def: str, entity_id: int, fields: Dict[str, Any] | None = None, projectid: int = 0, set_name: str = "ContractItem") -> dict:
    """
    Update ANY entity by id (repost same id with new fields).
    """
    ent = resolve_entity_def(entity_def)
    entity = {"id": int(entity_id), "entityDef": ent}
    if fields: entity.update(fields)
    resp = await _post_activity([entity], projectid=projectid, set_name=set_name)
    ctype = resp.headers.get("content-type", "")
    body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

@function_tool(strict_mode=False)
async def send_custom_item(entity: Dict[str, Any], projectid: int | None = None, set_name: str = "ContractItem") -> dict:
    """
    Fire a fully custom entity (dict) as-is. Ensures id=0 and entityDef present.
    """
    if not isinstance(entity, dict):
        return {"status": "error", "message": "entity must be a dict"}
    if not entity.get("entityDef"):
        return {"status": "error", "message": "entityDef is required"}
    entity = dict(entity)
    entity["id"] = 0
    pid = int(projectid if projectid is not None else entity.get("ProjectId") or 0)
    resp = await _post_activity([entity], projectid=pid, set_name=set_name)
    ctype = resp.headers.get("content-type", "")
    body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}
    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

# @function_tool
# async def find_project_id_by_name(name: str) -> dict:
#     """
#     Naive: query Projects and pick exact/closest Name match. Returns id or None.
#     """
#     qpayload = {"PropertyName": "Query", "EntityDef": resolve_entity_def("project")}
#     async with httpx.AsyncClient(timeout=20.0) as client:
#         resp = await client.post(QUERY_URL, headers=HEADERS_JSON(), json=qpayload)
#         body = resp.json() if "application/json" in (resp.headers.get("content-type","") or "") else {"text": resp.text}
#     if resp.status_code >= 400:
#         return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}

#     candidates = []
#     for key in ("entities", "results", "items"):
#         lst = body.get(key)
#         if isinstance(lst, list):
#             candidates = lst
#             break

#     best = None
#     name_lc = name.strip().lower()
#     for row in candidates:
#         nm = (row.get("Name") or row.get("name") or "").strip()
#         if nm.lower() == name_lc:
#             best = row
#             break
#         # keep first partial match as fallback
#         if not best and name_lc in nm.lower():
#             best = row
#     return {"status": "ok", "project_id": (best.get("Id") if isinstance(best, dict) else None), "match": best}

@function_tool(strict_mode=False)
async def upsert_entity(entity_def: str, match_fields: Dict[str, Any], write_fields: Dict[str, Any], projectid: int = 0) -> dict:
    """
    Simple upsert: Query, try to find first row where all match_fields match exactly.
    If match found → update_entity; else → create_entity(write_fields).
    """
    # Query all (you can optimize this with server-side filters if available)
    q = await query_entities(entity_def)
    if q.get("status") != "ok":
        return q

    def matches(row: Dict[str, Any], criteria: Dict[str, Any]) -> bool:
        for k, v in criteria.items():
            if row.get(k) != v:
                return False
        return True

    # Scan results for a match
    body = q["body"]
    rows = []
    for key in ("entities", "results", "items"):
        lst = body.get(key)
        if isinstance(lst, list):
            rows = lst
            break

    found = None
    for r in rows:
        if isinstance(r, dict) and matches(r, match_fields):
            found = r
            break

    if found and "Id" in found:
        ent_id = found["Id"]
        return await update_entity(entity_def, ent_id, write_fields, projectid)
    else:
        return await create_entity(entity_def, write_fields, projectid)

# ────────────────────────────────────────────────────────────────────────────────
# Models for convenience (optional)
# ────────────────────────────────────────────────────────────────────────────────
class ProjectItem(BaseModel):
    name: str = Field(..., description="Project Name")
    description: Optional[str] = None
    id: Optional[int] = None

SUPER_AGENT_INSTRUCTIONS = """You are the Kahua SuperAgent. Behave like ChatGPT and complete user tasks by calling tools.

Capabilities:
- Query any entity: call query_entities(entity_def or alias).
- Create any entity: call create_entity(entity_def or alias, fields, projectid).
- Update any entity by Id: call update_entity(entity_def or alias, entity_id, fields, projectid).
- Upsert any entity: call upsert_entity(entity_def or alias, match_fields, write_fields, projectid).
- Find project by name: call find_project_id_by_name(name).
- Send raw custom entity dicts: call send_custom_item(entity, projectid).

Rules:
- Accept either a friendly alias (e.g., "punch list") or a full entityDef (e.g., "kahua_AEC_PunchList.PunchListItem").
- Prefer exact field names when the user provides them; otherwise set only minimal safe fields (e.g., Subject, Description).
- If user names a project, resolve its Id first with find_project_id_by_name.
- Keep responses concise; return structured JSON summaries when appropriate.
- Do NOT ask the user for schema; rely on aliases or provided entityDef.

Current entityDef mappings:

contract: kahua_Contract.Contract
contract item: kahua_Contract.ContractItem
project: kahua_Project.Project
rfi: kahua_AEC_RFI.RFI
submittal: kahua_AEC_Submittal.SubmittalItem
change order: kahua_AEC_ChangeOrder.ChangeOrder
punch list: kahua_AEC_PunchList.PunchListItem
field observation: kahua_AEC_FieldObservation.FieldObservationItem
invoice: kahua_AEC_Invoice.Invoice
invoice item: kahua_AEC_Invoice.InvoiceItem

Short API overview:
Every post/creation/update/delete is sent to this API endpoint: https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/<projectid>/apps/image.png/activities/run with basic auth and a JSON payload.
Where <projectid> is the id of a particular project or 0, for a root creation. Do not be fooled by the kahua_AEC_RFI part, it is the same for all apps. This does not mean you are creating an RFI, its just a hardcoded endpoint.
This is the JSON payload format for a creation:
{
    "activity": {
        "PropertyName": "Activity",
        "Name": "Activity",
        "Flow": [
        {
            "PropertyName": "Iterate",
            "Set": "Activity",
            "New": {},
            "Existing": {
            }
        }
        ]
    },
    "listIdentifier": 0,
    "listId": 0,
    "sets": [
        {
        "name": "Activity",
        "entities": [
            {
            "id": <id>,
            "entityDef": <entity_def>,
            }
        ]
        }
    ]
    }
}
The only thing you ever need to change in this template is the <entity_def> and <id> fields (ALWAYS REQUIRED), as well as adding any additional fields you need inside the same dict as entitydef and id. id will be 0 when creating a new item, and will be the id of the existing item when updating. entity_def will be a Kahua entityDef of the item you are creating, such as kahua_AEC_RFI.FieldObservationItem.
Upon posting this JSON to the endpoint, you will receive a response with the entire schema of the new item, including the id field.
So, if there's an entity without any items you can just create a new one with id=0 and entityDef set to the entityDef of the entity you want to create to receive the entire schema of the new item, including the id field."""

super_agent = Agent(
    name="Kahua SuperAgent",
    handoff_description="Master of the Kahua API for query/creation/update/upsert.",
    model=OpenAIChatCompletionsModel(model=MODEL_DEPLOYMENT, openai_client=azure_client),
    instructions=SUPER_AGENT_INSTRUCTIONS,
    tools=[query_entities, create_entity, update_entity, upsert_entity, send_custom_item],
)



In [6]:
# ────────────────────────────────────────────────────────────────────────────────
# Demo run (adjust or remove)
# ────────────────────────────────────────────────────────────────────────────────
session = SQLiteSession("12345")

demo_prompt = (
"""
Create a punch list for me breaking my laptop in root. print the post request
"""
)

# Run inside notebooks
result = await Runner.run(super_agent, demo_prompt, session=session)
print("FINAL OUTPUT:\n", result.final_output)


2025-10-24 11:06:42,895 INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-10-24 11:06:43,576 INFO httpx: HTTP Request: POST https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/apps/kahua_AEC_RFI/activities/run "HTTP/1.1 200 OK"
2025-10-24 11:06:45,002 INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


FINAL OUTPUT:
 Here's the Post Request used to create the Punch List:

```json
{
    "activity": {
        "PropertyName": "Activity",
        "Name": "Activity",
        "Flow": [
            {
                "PropertyName": "Iterate",
                "Set": "Activity",
                "New": {},
                "Existing": {}
            }
        ]
    },
    "listIdentifier": 0,
    "listId": 0,
    "sets": [
        {
            "name": "Activity",
            "entities": [
                {
                    "id": 0,
                    "entityDef": "kahua_AEC_PunchList.PunchListItem",
                    "Subject": "Broken laptop",
                    "Description": "Laptop damaged due to accidental impact at work"
                }
            ]
        }
    ]
}
```

The punch list has been successfully created for the incident! Let me know if you need further assistance.
